Django提高04
====
**django.urls模块的实用函数**

如果您需要在代码中使用类似于url模板标记的内容，Django提供以下功能：

# reverse()

**`reverse(viewname, urlconf=None, args=None, kwargs=None, current_app=None)[source]`**

该函数的作用是逆向查找（解析）视图函数映射的url，查找出来的 url 值可以作为其它用作其它函数、表单等的参数。

viewname 可以是URL模式名称或可调用视图的对象。例如，给定以下url：

```
from news import views

path('archive/', views.archive, name='news-archive')
```

您可以使用下面两种方法的一种来逆向解析网址：

```
# using the named URL
reverse('news-archive')

# passing a callable object
# (This is discouraged because you can't reverse namespaced views this way.)
from news import views
reverse(views.archive)
```

如果URL接受参数，您可以在args中传递它们。例如：
```
from django.urls import reverse

def myview(request):
    return HttpResponseRedirect(reverse('arch-summary', args=[1945]))
    ```

可以传递 kwargs 代替 传递 args，例如：

```
>>> reverse('admin:app_list', kwargs={'app_label': 'auth'})
'/admin/auth/'
```

不能同时传递 args 和 kwargs 给reverse()。

如果不能匹配，则 reverse() 引发 NoReverseMatch异常。

reverse() 函数可以逆向解析 URL 的各种正则表达式模式，但不是每个可能的模式。目前的主要限制是模式不能包含使用竖线（“|”）字符的替代选择。您可以非常愉快地使用这些模式来匹配传入的URL并将它们发送到视图，但是您无法逆向解析这些模式。

current_app参数允许您向解析程序提供一个提示，指示当前正在执行的视图所属的应用程序。根据命名空间的URL解析策略，此current_app参数用作提示，以将应用程序命名空间解析为特定应用程序实例上的URL。

urlconf参数是URLconf模块，其中包含用于逆向解析的URL模式。默认情况下，使用当前线程的根URLconf。

**注意**

reverse() 返回的字符串已经是 urlquoted。 例如：

```
>>> reverse（'cities'，args = ['Orléans']）
'... / ORL％C3％A9ans/'
```
将进一步的编码（ encoding ）（例如urllib.parse.quote（））应用于 reverse() 的输出可能会产生不良结果。

# reverse_lazy()

reverse()的延缓估值版本。

**`reverse_lazy（viewname，urlconf = None，args = None，kwargs = None，current_app = None）`**

当在加载项目的 URLConf 之前需要使用URL的逆向解析时，它非常有用。需要此功能的一些常见情况是：

- 提供逆向 URL 作为基于通用类的视图的url属性。
- 为装饰器提供逆向 URL（例如django.contrib.auth.decorators.permission_required（）装饰器的login_url参数）。
- 提供逆向 URL 作为函数签名中参数的默认值。

# resolve()

resolve（）函数可用于解析相应视图函数的URL路径。 它有以下签名：

** `resolve（path，urlconf = None）` **

path 是您要解析的URL路径。与reverse() 一样，您不必担心urlconf参数。该函数返回ResolverMatch对象，该对象允许您访问有关已解析URL的各种元数据。

如果URL未解析，则该函数会引发Resolver404异常（Http404的子类）。

## ResolverMatch 类

**·class ResolverMatch·**

**func**

用来服务URL的视图函数。

**args**

这个参数将传递给视图函数，从URL中解析出这个参数。

**kwargs**

这个关键字参数将传递给视图函数，从URL中解析出这个参数。

**url_name**

与URL匹配的URL模式的名称。

**app_name**

与URL匹配的URL模式的应用程序命名空间。

**app_name**

与URL匹配的URL模式的完整应用程序命名空间中的各个命名空间组件的列表。 例如，如果 app_name 是'foo：bar'，那么app_names将是['foo'，'bar']。

**namespace**

与URL匹配的URL模式的实例命名空间。

**namespaces**

与URL匹配的URL模式的完整实例命名空间中的各个命名空间组件的列表。 即，如果命名空间是foo：bar，那么命名空间将是['foo'，'bar']。

**view_name**

与URL匹配的视图的名称，包括名称空间（如果有）。

然后可以询问 ResolverMatch 对象以提供有关与URL匹配的URL模式的信息：

```
# Resolve a URL
match = resolve('/some/path/')
# Print the URL pattern that matches the URL
print(match.url_name)
```

ResolverMatch对象也可以分配给三元组：

`func, args, kwargs = resolve('/some/path/')`

resolve（）的一个可能用途是测试视图在重定向到它之前是否会引发Http404错误：

```
from urllib.parse import urlparse
from django.urls import resolve
from django.http import Http404, HttpResponseRedirect

def myview(request):
    next = request.META.get('HTTP_REFERER', None) or '/'
    response = HttpResponseRedirect(next)

    # modify the request and response as required, e.g. change locale
    # and set corresponding locale cookie

    view, args, kwargs = resolve(urlparse(next)[2])
    kwargs['request'] = request
    try:
        view(*args, **kwargs)
    except Http404:
        return HttpResponseRedirect('/')
    return response
    ```

# get_script_prefix()
g
**`et_script_prefix()[source]`**

通常，您应该始终使用reverse（）来定义应用程序中的URL。 但是，如果您的应用程序构建了URL层次结构本身的一部分，您可能偶尔需要生成URL。 在这种情况下，您需要能够在其Web服务器中找到Django项目的基本URL（通常，reverse（）会为您处理此问题）。 在这种情况下，您可以调用get_script_prefix（），它将返回Django项目的URL的脚本前缀部分。 如果您的Django项目位于其Web服务器的根目录，则始终为“/”。